In [19]:
import numpy as np
import pandas as pd
import os
import re
import math
import random
import pickle

import matplotlib.dates as mdates
from datetime import time
xformatter = mdates.DateFormatter('%H:%M')  # for time axis plots
import datetime
from dateutil.parser import parse
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline
from matplotlib import style
style.use('seaborn-whitegrid')
from tensorflow.keras.optimizers import Adam, RMSprop, SGD, Adamax
from pandas.tseries.frequencies import to_offset
from pickle import load,dump
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest,f_regression
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM,Bidirectional,BatchNormalization, Dropout
from keras.models import load_model
from keras.callbacks import EarlyStopping
from geneticalgorithm import geneticalgorithm as ga
from keras.backend import clear_session
import warnings
warnings.filterwarnings('ignore')

In [20]:
FORMAT_TIME='%Y-%m-%d %H:%M:%S'
START_TIME_DAY = time(5,0,0)
END_TIME_DAY = time(18,30,0)

In [21]:
TIME_1_STEP = 15 # minute
step_lag_1_day = 24*60//TIME_1_STEP
steps_2hours = 60*2//TIME_1_STEP

In [22]:
#Define optimization
activation_list = ['linear','relu', 'tanh','sigmoid']
optimizer_list = [Adam, RMSprop, SGD, Adamax]
range_neurons = [8,100]
selectors = []
algorithm_param = {
    'max_num_iteration': 5,
    'population_size': 5,
    'mutation_probability': 0.1,
    'elit_ratio': 0.2,
    'crossover_probability': 0.5,
    'parents_portion': 0,
    'crossover_type': 'uniform',
    'max_iteration_without_improv': 2
}

In [23]:
# Nhap du lieu train data
def import_train_data(path_file_train):
    df = pd.read_csv(path_file_train)
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp']
                                      )
    df = df.set_index('TimeStamp')
    return df

In [24]:
def resample_df(df, resample_time, time_col='TimeStamp'):
    """
    resample_time: `minute`
    """
    resample_df = df.copy()
    if resample_time >= 30:
        resample_df = resample_df.set_index(
            resample_df[time_col] - to_offset(str(resample_time//2)+"min"))
    resample_df = resample_df.resample(str(resample_time)+'min', label='right').mean()
    return resample_df

In [25]:
all_plant = import_train_data('./LN2_training.csv')
# all_plant = all_plant.reset_index()
all_plant = resample_df(all_plant, resample_time = 15, time_col='TimeStamp')

In [26]:
all_plant

,TotW,I_GHI,T
TimeStamp,,,
2021-04-01 00:15:00,0.0,0.0,28.258230
2021-04-01 00:30:00,0.0,0.0,28.100050
2021-04-01 00:45:00,0.0,0.0,27.941867
2021-04-01 01:00:00,0.0,0.0,27.783687
2021-04-01 01:15:00,0.0,0.0,27.658317
...,...,...,...
2022-08-31 23:00:00,0.0,0.0,25.619470
2022-08-31 23:15:00,0.0,0.0,25.582193
2022-08-31 23:30:00,0.0,0.0,25.544917


In [27]:
# Split train, val data theo ti le 8/2
def train_valid_split(df, split_ratio=[0.8, 0.2]):
    train_ratio, valid_ratio = split_ratio
    assert train_ratio + valid_ratio  == 1.0
    n_df = len(df)
    # Train / Validation  Split
    train_split = int(n_df * train_ratio)
    valid_split = int(n_df * (train_ratio + valid_ratio))

    train = df[:train_split]
    val = df[train_split:valid_split]

    print(f'Train set: {len(train)} ')
    print(f'Validation set: {len(val)} ')

    return train, val 

In [28]:
def num_step_1hour(df):
    """
    Get number step of 1 hours
    """
    step_hours = None
    if type(df.index) == pd.core.indexes.datetimes.DatetimeIndex:
        time_1step = int((df.index[1] - df.index[0]) /
                         np.timedelta64(1, 'm'))  # minute
        step_hours = 60 // time_1step
    return step_hours

In [29]:
def make_data_supervised(dt, num_pre_around=5, num_day_pre=3):
    step_lag_1_day = num_step_1hour(dt)*24
    dt_lag = pd.DataFrame()
    for col in dt.columns:
        for day_pre in range(num_pre_around+1):
            if day_pre == 0:
                dt_lag[col+'(t)'] = dt[col]
            else:
                dt_lag[col+'(t-'+str(day_pre)+')'] = dt[col].shift(day_pre)
        for day_pre in range(1, num_day_pre):
            step_lag = step_lag_1_day*day_pre
            for lag in range(num_pre_around+1):
                dt_lag[col+'(t-'+str(step_lag+lag) +
                       ')'] = dt[col].shift(step_lag+lag)
                dt_lag[col+'(t-'+str(step_lag-lag) +
                       ')'] = dt[col].shift(step_lag-lag)
    dt_lag = dt_lag.dropna()
    dt_lag['hour'] = dt_lag.index.hour
    dt_lag['day'] = dt_lag.index.day
    dt_lag['day_of_week'] = dt_lag.index.dayofweek
    dt_lag['month'] = dt_lag.index.month
    dt_lag['day_of_year'] = dt_lag.index.dayofyear
    return dt_lag

In [30]:
def select_Kbest(train,score_f):
    X_select = train[train.columns[1:]]
    y_select = train['TotW(t)']
    bestfeatures = SelectKBest(score_func=score_f, k='all')
    fit = bestfeatures.fit(X_select,y_select)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(X_select.columns)
    featureScores = pd.concat([dfcolumns,dfscores],axis=1)
    featureScores.columns = ['Specs','Score']  
    featureScores = featureScores.sort_values(by='Score', ascending=False)

    return featureScores

In [31]:
def make_model(train, val, ga_result):
    train_X, train_y = train.values[:, 1:], train.values[:, :1]
    train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])

    val_X, val_y = val.values[:, 1:], val.values[:, :1]
    val_X = val_X.reshape(val_X.shape[0], 1, val_X.shape[1])

    model = Sequential()
    model.add(
        Bidirectional(LSTM(ga_result['num_feature'],
             activation=ga_result['activations'],
             input_shape=(train_X.shape[1], train_X.shape[2])),merge_mode= 'ave'))
    
    # model.add(BatchNormalization())
    # model.add(Dropout(0.2))   
    # model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mae',
                  optimizer=ga_result['optimizers'](learning_rate = 0.001),
                  metrics=['mse', 'mae', 'cosine_proximity'])
    # fit network
    history = model.fit(train_X,
                        train_y,
                        epochs=100,
                        batch_size=50,
                        validation_data=(val_X, val_y),
                        verbose=1,
                        shuffle=False,
                        callbacks=EarlyStopping(monitor='val_loss',
                                                patience=15,
                                                restore_best_weights=True))
    fig, ax = plt.subplots(figsize=(16, 8))
    fig.suptitle('Loss', y=0.93)
    ax.plot(history.history['mae'], label='train')
    ax.plot(history.history['val_mae'], label='val')
    ax.set_title('mae')
    ax.legend(loc='upper right')
    return model

In [32]:
def GA_result(ga_model):
    best_neurons = (int(ga_model.best_variable[0]))
    best_activation = (int(ga_model.best_variable[1]))
    best_optimizer = (int(ga_model.best_variable[2]))
    return {
        'activations': activation_list[best_activation],
        'optimizers': optimizer_list[best_optimizer],
        'neurons': best_neurons,
        'num_feature': int(ga_model.best_variable[3])
    }

In [33]:
def get_or_create_path(path):
    if os.path.isdir(path) is False:
        os.mkdir(path)

In [34]:
def scale_func(train, val, path_scale):
    get_or_create_path(path_scale)
    train_scale_df = pd.DataFrame(index=train.index)
    val_scale_df = pd.DataFrame(index=val.index)
    
    for col in train.columns:
        scaler = MinMaxScaler()
        train_scale_df[col] = scaler.fit_transform(train[col].values.reshape(-1,1))[:,0]
        val_scale_df[col] = scaler.transform(val[col].values.reshape(-1,1))[:,0]
        pickle.dump(scaler, open(path_scale + col+'.pkl','ab+'))
    return train_scale_df,val_scale_df

In [35]:
#def tong
def LSTM_generetic(df, path_model, path_scale, path_feature):
    #Hàm evaluate
    def evaluate(gene):
        g = [int(i) for i in gene]
        train_X_ = train_X.copy()
        val_X_ = val_X.copy()
        train_y_ = train_y.copy()
        val_y_ = val_y.copy()
        selector = SelectKBest(score_func=f_regression, k=g[3])
        train_X_ = train_X_.reshape(train_X_.shape[0], train_X_.shape[2])
        val_X_ = val_X_.reshape(val_X_.shape[0], val_X_.shape[2])
        train_X_ = selector.fit_transform(train_X_, train_y_)
        val_X_ = selector.transform(val_X_)
        train_X_ = train_X_.reshape(train_X_.shape[0],1,train_X_.shape[1])
        val_X_ = val_X_.reshape(val_X_.shape[0],1,val_X_.shape[1])
        neurons = g[0]
        activation = activation_list[g[1]]
        optimizer = optimizer_list[g[2]] 
        print('\nNumber of neurons: ', neurons,
              ', activation function: ', activation,
              ', optimizer function: ', optimizer,
              ', features: ', g[3])
        clear_session()
        model = Sequential()
        model.add(Bidirectional(LSTM(neurons, activation=activation, 
                                     input_shape=(train_X_.shape[1], train_X_.shape[2]),return_sequences=True)))
        model.add(LSTM(64))
        # model.add(BatchNormalization())
        # model.add(Dropout(0.2))   
        # model.add(Dense(16, activation='relu'))
        model.add(Dense(1))
        model.compile(loss='mae', optimizer=optimizer(learning_rate = 0.001))
        history = model.fit(train_X_, train_y_,validation_data=(val_X_, val_y_),callbacks = EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True), 
            epochs=100, batch_size=50, verbose=0,shuffle=False)
        print('val_loss: ', min(history.history['val_loss']))
        return min(history.history['val_loss'])
        
    
    train_solar, val_solar = train_valid_split(df,split_ratio=[0.8, 0.2]) # split train/val
    train_scale, val_scale = scale_func(train_solar, val_solar, path_scale)

    train_scaler_lag = make_data_supervised(train_scale)
    val_scaler_lag = make_data_supervised(val_scale)
    
    col_analysis = list(train_scaler_lag)
    train_solar = train_scaler_lag[col_analysis].copy()
    val_solar = val_scaler_lag[col_analysis].copy()
    
    featureScores = select_Kbest(train_solar[col_analysis],f_regression)
    featureScores.to_csv('./LN2_4H_kbest.csv')
    
    train_X, train_y= train_solar.values[:, 1:],train_solar.values[:, :1]
    train_X = train_X.reshape(train_X.shape[0], 1, train_X.shape[1])
    val_X, val_y= val_solar.values[:, 1:],val_solar.values[:, :1]
    val_X = val_X.reshape(val_X.shape[0], 1, val_X.shape[1])
    
    best_activation, best_optimizer, best_neurons, selectors = [], [], [], []
    varbound = np.array([range_neurons, [0, 3], [0, 3], [1, train_X.shape[2]]])
    ga_model = ga(function=evaluate,
                dimension=4,
                variable_type='int',
                function_timeout=10000,
                variable_boundaries=varbound,
                convergence_curve=False,
                algorithm_parameters=algorithm_param)
    ga_model.run()
    
    ga_result = GA_result(ga_model)
    best_feature = list(featureScores['Specs'][:ga_result['num_feature']])
    pickle.dump(best_feature, open(path_feature + 'best_feature.pkl','wb'))
    train_solar = train_scaler_lag[['TotW(t)']+best_feature]
    val_solar = val_scaler_lag[['TotW(t)']+best_feature]
     
    model = make_model(train_solar,val_solar,ga_result)
    model.save(path_model)
    
    return featureScores, ga_result, model

In [36]:
LSTM_generetic(all_plant,'./LN2/4H.h5','./LN2/','./LN2/')

Train set: 39782 
Validation set: 9946 

Number of neurons:  40 , activation function:  sigmoid , optimizer function:  <class 'keras.optimizer_v2.adamax.Adamax'> , features:  30
val_loss:  0.010581242851912975

Number of neurons:  22 , activation function:  tanh , optimizer function:  <class 'keras.optimizer_v2.gradient_descent.SGD'> , features:  53
val_loss:  0.03555513918399811

Number of neurons:  47 , activation function:  sigmoid , optimizer function:  <class 'keras.optimizer_v2.adamax.Adamax'> , features:  5
val_loss:  0.01090223528444767

Number of neurons:  66 , activation function:  sigmoid , optimizer function:  <class 'keras.optimizer_v2.gradient_descent.SGD'> , features:  26
val_loss:  0.04634219408035278

Number of neurons:  39 , activation function:  tanh , optimizer function:  <class 'keras.optimizer_v2.adam.Adam'> , features:  51
val_loss:  0.010760901495814323
||||||||||________________________________________ 20.0% GA is running...
Number of neurons:  40 , activation 